# FINAL PROJECT - OCCASION VEHICLES PRICE ESTIMATOR
___
#### MASTER IN DATA SCIENCE - KSCHOOL - 2016/17
#### KOLDO PINA ORTIZ
____

## Motivation

The aim of the present work is to get a good estimator of the price of a second-hand vehicle, based on the prices of the second-hand market

## Action Plan

To reach our goal, we will follow the the next steps:

1. ***Scrape*** website, **motos.net** to obtain the ***data***.
2. ***Clean*** the ***data***.
2. ***Merge*** some ***data***.
3. ***Train*** various models.
4. Compare the metrics and choose the model with the best one.
5. Create a flask web server with the app.

## Scrape

We are going to scrape each website separately.
To this end, we have developed two python scripts called ***scraper_motos.py*** and ***scrapers_coches.py***.
Both return a dataframe.

You can import them to the notebook.

In [5]:
#Call the scraper_motos function to scrape motos.net and create a csv file with the raw data
#from scraper_motos import scraper_motos
#scraper_motos().to_csv('motos_raw_data.csv', index = False, header = True, encoding = 'utf-8')

In [ ]:
#Start time: 2017-05-14 20:41:19.733669
#num_ads 26161
#End time: 2017-05-14 23:58:13.605710

In [2]:
#from scraper_coches import Scraper_coches
#Scraper_coches().to_csv('coches_raw_data.csv', index = False, header = True, encoding = 'utf-8')

In [ ]:
#Start time: 2017-05-03 19:44:43.260000s
#num_pages 5365
#End time: 2017-05-04 12:12:29.699000

In [1]:
import pandas as pd
df_motos_raw = pd.read_csv('motos_raw_data.csv')
df_motos_raw.shape

(26081, 10)

In [2]:
df_motos_raw.head()

,city,brand,model,type,cc,color,km,year,price,url
0,Granada,TRIUMPH,SPEED TRIPLE,Naked,1050.0,Amarillo,32000.0,2006,4500,http://motos.net/ocasion/triumph/speed_triple/...
1,Vizcaya,BMW,R 1200 GS 98cv,Trail,1170.0,Blanco,74200.0,2007,8500,http://motos.net/ocasion/bmw/r_1200_gs_98cv/20...
2,Asturias,BMW,R 1200 GS,Trail,1170.0,Rojo,79000.0,2007,7500,http://motos.net/ocasion/bmw/r_1200_gs/2007-en...
3,Sevilla,HARLEY DAVIDSON,Sportster 883 XLH 53,Custom,883.0,Negro,22000.0,2003,3500,http://motos.net/ocasion/harley_davidson/sport...
4,Valencia,KYMCO,Super Dink 125i,Scooter 125cc,125.0,NaN,33000.0,2012,1950,http://motos.net/ocasion/kymco/super_dink_125i...


## Data cleaning

In [3]:
# Convert the dataframe to lower case
df_motos_raw = df_motos_raw.apply(lambda x: x.astype(str).str.lower())

In [4]:
#Join the words in the 'model' and 'type' fields with an underscore
df_motos_raw['model'] = df_motos_raw['model'].str.replace(' ', '_')
df_motos_raw['type'] = df_motos_raw['type'].str.replace(' ', '_')

In [5]:
# Looking for duplicates
df_motos_raw['is_duplicated'] = df_motos_raw.duplicated()
duplicates = df_motos_raw['is_duplicated'].sum()
print '%d duplicates' %duplicates

61 duplicates


In [6]:
# Removing duplicates and delete 'is_duplicated' column
df_motos_raw = df_motos_raw.loc[df_motos_raw['is_duplicated']==False]
df_motos_raw = df_motos_raw.drop('is_duplicated', 1)
df_motos_raw.shape

(26020, 10)

In [7]:
# Lets investigate column by column the NaNs we have in the dataframe
for column in df_motos_raw.columns:
    n_nan = df_motos_raw[column]=='nan'
    print column + " %d -- > %f" %(n_nan.sum(), (n_nan.sum()*1.0)/df_motos_raw.shape[0]*100)

city 0 -- > 0.000000
brand 0 -- > 0.000000
model 0 -- > 0.000000
type 0 -- > 0.000000
cc 97 -- > 0.372790
color 7094 -- > 27.263643
km 1624 -- > 6.241353
year 0 -- > 0.000000
price 0 -- > 0.000000
url 0 -- > 0.000000


In [8]:
# Lets investigate the unique values we have in the columns
for column in ['city', 'brand', 'model', 'type', 'color', 'year']:
    column_uv = df_motos_raw[column].unique()
    print column + " --> " + "%d unique values" %len(sorted(column_uv))

city --> 52 unique values
brand --> 146 unique values
model --> 2989 unique values
type --> 15 unique values
color --> 1932 unique values
year --> 48 unique values


## Merge some data

In [9]:
#In order to calculate our first metric, we will use the following columns:
# "lon" and "lat" : These are the longitude and latitude of the corresponding city. We will add them later.
# "brand", "model", "type", "year"

In [10]:
# Calculating the longitude and latitude of the cities
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [11]:
cities = df_motos_raw['city'].unique()
locations_rows = []
for city in cities:
    location = geolocator.geocode([city], timeout = 15)
    locations_rows.append([city, location.latitude, location.longitude])
#Save into a csv
df_locations = pd.DataFrame(locations_rows, columns = ['city', 'lat', 'lon'])
df_locations.to_csv('locations_coords.csv', index = False)

In [12]:
df_locations.head(3)

,city,lat,lon
0,granada,37.183020,-3.602192
1,vizcaya,43.238264,-2.852207
2,asturias,43.271563,-5.853946


In [13]:
# Merge df_locations with df_motos_raw
df_motos_raw_coord = pd.merge(df_motos_raw, df_locations, on = 'city')
#Save into a csv
df_motos_raw_coord.to_csv('df_motos_raw_coord.csv', index = False)

In [16]:
# We have created two csv files with a rank for the motos brands and types
# called rank_motos_brands.csv and rank_motos_types.csv

# With the first one, rank_motos_brands.csv, we are gint to create another  column, with a score to the corresponding brand
df_rank_moto_brand = pd.read_csv('rank_moto_brands.csv', sep=';')
df_motos_raw_coord_brand = pd.merge(df_motos_raw_coord, df_rank_moto_brand, on = 'brand', how = 'left')
# Save into a csv
df_motos_raw_coord_brand.to_csv('df_motos_coord_brand.csv', index = False)

# With the second one, rank_moto_types.csv, we are going to create another column, with a score to the corresponding type
df_rank_moto_type = pd.read_csv('rank_moto_types.csv', sep=';')
df_motos_raw_coord_brand_types = pd.merge(df_motos_raw_coord_brand, df_rank_moto_type, on = 'type', how = 'left')
#Save into a csv
df_motos_raw_coord_brand_types.to_csv('df_motos_raw_coord_brand_type.csv', index = False)


In [18]:
df_motos_raw_coord_brand_types.head(3)

,city,brand,model,type,cc,color,km,year,price,url,lat,lon,brand_score,type_score
0,granada,triumph,speed_triple,naked,1050.0,amarillo,32000.0,2006,4500,http://motos.net/ocasion/triumph/speed_triple/...,37.18302,-3.602192,120.0,15.0
1,granada,triumph,tiger_800,trail,799.0,negro,39200.0,2011,6200,http://motos.net/ocasion/triumph/tiger_800/201...,37.18302,-3.602192,120.0,14.0
2,granada,kawasaki,vn_900_classic_special_edition,custom,903.0,negro,32000.0,2011,5000,http://motos.net/ocasion/kawasaki/vn_900_class...,37.18302,-3.602192,378.0,13.0


In [19]:
df_motos_raw_coord_brand_types.shape

(26020, 14)